In [ ]:
# SISTEMA
%%bash
set -e

apt-get update
apt-get install -y zip unzip git openjdk-17-jdk python3-dev python3-pip

In [ ]:
# Instalar Buildozer e Cython
%%bash
pip install -U Cython==0.29.36
pip install -U buildozer
pip install -U setuptools wheel


In [ ]:
# path de correção do buildozer
%%bash
FILE=$(python3 - << 'EOF'
import buildozer, inspect, os
import buildozer.__init__ as b
path = inspect.getsourcefile(b)
print(path)
EOF
)

echo "Editando: $FILE"

sed -i '/def check_root/s/$/\n        return/' "$FILE"




Editando: /usr/local/lib/python3.12/dist-packages/buildozer/__init__.py


In [19]:
# CRIAR PROJETO SIMPLES
%%bash
set -e

mkdir -p /content/app
cat > /content/app/main.py << 'EOF'
from kivy.app import App
from kivy.uix.label import Label

class Teste(App):
    def build(self):
        return Label(text="Hello Kivy no Colab!")

Teste().run()
EOF


In [ ]:
# gerar guildozer spec
%%bash
cd /content/app
buildozer init

File buildozer.spec created, ready to customize!


In [ ]:
# Instalar Android SDK + build-tools + AIDL
%%bash
set -e

export ANDROID_HOME=$HOME/.buildozer/android/platform/android-sdk
mkdir -p $ANDROID_HOME

# 0 — Limpar instalações quebradas do cmdline-tools
rm -rf $ANDROID_HOME/cmdline-tools

# 1 — Baixar commandlinetools oficial
cd /content
wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip

# 2 — Extrair
unzip -qo commandlinetools-linux-9477386_latest.zip -d $ANDROID_HOME

# Agora o Google coloca tudo dentro de: cmdline-tools/
# Mas o sdkmanager exige a estrutura: cmdline-tools/latest/

mkdir -p $ANDROID_HOME/cmdline-tools/latest

# 3 — Mover APENAS os arquivos internos, não a pasta inteira
mv $ANDROID_HOME/cmdline-tools/* $ANDROID_HOME/cmdline-tools/latest/ 2>/dev/null || true

# 4 — PATH correto
export PATH=$ANDROID_HOME/cmdline-tools/latest/bin:$PATH

# 5 — Instalar pacotes
yes | sdkmanager --sdk_root=$ANDROID_HOME \
    "platform-tools" \
    "platforms;android-33" \
    "build-tools;33.0.2" \
    "cmdline-tools;latest" \
    "extras;google;m2repository" \
    "extras;android;m2repository"

# 6 — Verificar AIDL
ls $ANDROID_HOME/build-tools/33.0.2/




In [ ]:
# enganar buildozer
%%bash
set -e

export ANDROID_HOME=$HOME/.buildozer/android/platform/android-sdk

# Criar caminho antigo esperado pelo buildozer
mkdir -p $ANDROID_HOME/tools/bin

# Criar link simbólico para o sdkmanager real
ln -sf $ANDROID_HOME/cmdline-tools/latest/bin/sdkmanager \
       $ANDROID_HOME/tools/bin/sdkmanager

# Só pra validar
ls -l $ANDROID_HOME/tools/bin


total 4
lrwxrwxrwx 1 root root 81 Nov 22 01:00 sdkmanager -> /root/.buildozer/android/platform/android-sdk/cmdline-tools/latest/bin/sdkmanager


In [ ]:
# fix
%%bash
set -e

export ANDROID_HOME=$HOME/.buildozer/android/platform/android-sdk

# criar estrutura antiga esperada pelo buildozer
mkdir -p $ANDROID_HOME/tools/bin

# criar symlink apontando para o sdkmanager real
ln -sf $ANDROID_HOME/cmdline-tools/latest/bin/sdkmanager \
       $ANDROID_HOME/tools/bin/sdkmanager

ln -sf $ANDROID_HOME/cmdline-tools/latest/bin/avdmanager \
       $ANDROID_HOME/tools/bin/avdmanager

echo "OK — sdkmanager agora está em:"
ls -l $ANDROID_HOME/tools/bin


OK — sdkmanager agora está em:
total 8
lrwxrwxrwx 1 root root 81 Nov 22 01:11 avdmanager -> /root/.buildozer/android/platform/android-sdk/cmdline-tools/latest/bin/avdmanager
lrwxrwxrwx 1 root root 81 Nov 22 01:11 sdkmanager -> /root/.buildozer/android/platform/android-sdk/cmdline-tools/latest/bin/sdkmanager


In [ ]:
# install autotools
%%bash
set -e

apt-get update

# ferramentas essenciais para compilar libffi, openssl, sqlite etc.
apt-get install -y \
    autoconf \
    automake \
    libtool \
    pkg-config \
    build-essential

echo "Autotools instalados!"


In [ ]:
# rodar build sem log
%%bash
cd /content/app

export BUILDOZER_ALLOW_ROOT=1
export ANDROID_HOME=$HOME/.buildozer/android/platform/android-sdk

# PATH correto
export PATH=$ANDROID_HOME/cmdline-tools/latest/bin:$PATH
export PATH=$ANDROID_HOME/platform-tools:$PATH
export PATH=$ANDROID_HOME/build-tools/33.0.2:$PATH
export PATH=$ANDROID_HOME/tools/bin:$PATH

# Executa buildozer com log AO VIVO
stdbuf -o0 buildozer -v android debug





Process is interrupted.


In [ ]:
# rodar build com log
import subprocess
import os

os.chdir("/content/app")

os.environ["BUILDOZER_ALLOW_ROOT"] = "1"
os.environ["ANDROID_HOME"] = "/root/.buildozer/android/platform/android-sdk"

# PATHs
os.environ["PATH"] += ":/root/.buildozer/android/platform/android-sdk/cmdline-tools/latest/bin"
os.environ["PATH"] += ":/root/.buildozer/android/platform/android-sdk/platform-tools"
os.environ["PATH"] += ":/root/.buildozer/android/platform/android-sdk/build-tools/33.0.2"
os.environ["PATH"] += ":/root/.buildozer/android/platform/android-sdk/tools/bin"

# Comando do buildozer
cmd = ["buildozer", "-v", "android", "debug"]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

# Imprime linha por linha em tempo real
for line in process.stdout:
    print(line, end="")
